In [5]:
import csv
import os



def calcmean(alist):
    mean = sum(alist)/len(alist)
    return mean

def calcmedian(alist):
    sortedlist=sorted(alist)
    midt=len(sortedlist)//2
    if len(sortedlist) % 2 == 0:
        return sum(sortedlist[midt-1:midt+1])/2.0
    else:
        return sortedlist[midt]

def checkNumbers(string):
    return any(char.isdigit() for char in string)


def participantCalc(filename):
    time=[]
    stim=[]
    resp=[]
    rtlistRare=[]
    rtlistFreq=[]
    holder1=0
    holder2=0
    accuracyRare=[]
    accuracyFreq=[]
    typeTrial=0
    with open(filename, 'r') as p:
        for i in range(1, 6):
            next(p, None)
        # easier way to skip the header probably exists
        reader=csv.reader(p, delimiter='\t')
        for row in reader:
            currStim=str(row[2])
            if currStim.startswith('STIM'):
                if checkNumbers(currStim) == True:
                    holder1=float(row[0])
                    typeTrial=1
                    #type 1 is the rare type
                else:
                        holder2=float(row[0])
                        typeTrial=2
                        #type 2 is the freq type
            elif currStim.startswith('RESP'):
                if typeTrial==1:
                    rtlistRare.append(float(row[0])-holder1)
                    if currStim.endswith('2'):
                        accuracyRare.append(1)
                    else:
                            accuracyRare.append(0)
                elif typeTrial==2:
                    rtlistFreq.append(float(row[0])-holder2)
                    if currStim.endswith('1'):
                        accuracyFreq.append(1)
                    else:
                            accuracyFreq.append(0)
    accuracyRareP = (float(sum(accuracyRare))/float(len(accuracyRare)))
    accuracyFreqP = (float(sum(accuracyFreq))/float(len(accuracyFreq)))

    return({'Freq mean':calcmean(rtlistFreq), 
            'Freq median':calcmedian(rtlistFreq), 
            'Freq accuracy':accuracyFreqP,
           'Rare mean':calcmean(rtlistRare),
           'Rare median':calcmedian(rtlistRare),
           'Rare accuracy':accuracyRareP})



#get list of all logs

logsList=os.listdir('logs')
scrubList=[]
for names in logsList:
    if names.endswith('.log'):
        scrubList.append(names)
#and add back logs/
realList=['logs/' + x for x in scrubList]

#now go through every log file, determine if patient/control
# and write that stuff to a csv file
#maybe I'll just read that csv file as well
patientOverview={}
with open('logs/subj_codes.csv', 'r') as r:
    reader2=csv.reader(r, delimiter=';')
    for row in reader2:
        patientOverview[row[0]]=row[1]
        

#I'm making quite a few assumptions here, 
#that for instance the participant code is always 8 characters
# And also that there are no missed trials or missing data.


# now try to collect this stuff, and write it.



with open('assignmentOle.csv', 'w') as csvfile:
    theWriter = csv.writer(csvfile, delimiter=',')
    #LETS HARDCODE A BIT OF HEADER
    theWriter.writerow(['Subjid', 'Group', 'Cond', 'Mean RT', 'Median RT', 'Accuracy'])
    for filename in realList:
        currOutput=participantCalc(filename)
        currId=filename[5:13]
        currType=patientOverview[currId]
        
        theWriter.writerow([currId, currType, 'Freq', 
                           currOutput['Freq mean'],
                          currOutput['Freq median'],
                          currOutput['Freq accuracy']])
        theWriter.writerow([currId, currType, 'Rare', 
                           currOutput['Rare mean'],
                          currOutput['Rare median'],
                          currOutput['Rare accuracy']])
        
    